In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest
import seaborn as sns
from scipy import stats
# Para QQ-plot
import statsmodels.api as sm

In [0]:
df = pd.read_csv('/Volumes/data_catalog_01_d/bronze/vol_data/nhanes_2015_2016_v2.csv')
alpha = 0.05 # nível de significância de 5% com nível de confiabilidade de 95%


### **1. Teste de Hipótese: Diferença de Altura entre fumantes e não fumantes**

**Pergunta:** Teste a hipótese de que a altura média (Altura_cm) de adultos que já fumaram ≥100 cigarros (variável JaFumou100Cigarros) é maior do que a de adultos que nunca fumaram.
Formule:

H0: μ_fumantes ≤ μ_não_fumantes 

HA: μ_fumantes > μ_não_fumantes

Use t-test unilateral e escolha um nível de significância justificando a sua escolha diante do contexto dos dados.

**Entregáveis:**
- Estatística t
- p-valor unilateral
- Conclusão interpretada no contexto da Aula 4


In [0]:
# Cria uma cópia do DataFrame original para evitar alterações nos dados originais
df_1 = df.copy()

# Remove linhas com valores nulos na coluna 'Altura_cm'
df_1.dropna(subset='Altura_cm', inplace=True)

# Seleciona as alturas dos indivíduos que já fumaram 100 cigarros
fuma_cig = df_1[df_1['JaFumou100Cigarros'] == 1]['Altura_cm']

# Seleciona as alturas dos indivíduos que nunca fumaram 100 cigarros
n_fuma_cig = df_1[df_1['JaFumou100Cigarros'] == 2]['Altura_cm']

# Realiza o teste t independente para comparar as médias das alturas dos dois grupos
t_stat, p_value = stats.ttest_ind(fuma_cig, n_fuma_cig, equal_var=False)

# Calcula o p-valor unilateral considerando a direção da hipótese alternativa
p_value_unilateral = p_value / 2 if t_stat > 0 else 1 - (p_value / 2)

# Exibe o valor de p_value
print(f"p-value: {p_value:.3f}")

# Exibe o valor da estatística t
print(f"t estatístico: {t_stat:.3f}")

# Exibe o p-valor unilateral do teste
print(f"p-valor unilateral: {p_value_unilateral:.5f}")

# Verifica se o p-valor é menor que o nível de significância e interpreta o resultado
if p_value_unilateral < alpha:
    print("HA (hipótese alternativa) - A altura média dos fumantes é maior que a dos não fumantes.")
else:
    print("H0 (hipótese nula) - Não há evidência de que a altura média dos fumantes seja maior.")

### 2. Teste de Normalidade (Shapiro-Wilk) para Pressão Diastólica por Grupo de Sexo

**Pergunta:** Verifique se a variável PressaoDiastolica_1 segue distribuição normal separadamente
para homens e para mulheres.

**Entregáveis:**
- reporte a estatística do teste
- p-valor
- conclua se H0 ("dados são normais") deve ser rejeitada.

**Responda:**
- Qual dos dois grupos demonstra maior desvio da normalidade? Explique.

In [0]:

df_2 = df.copy()

# Remove linhas com valores nulos na coluna 'PressaoDiastolica_1'
df_2.dropna(subset='PressaoDiastolica_1', inplace=True)

# Seleciona os valores de pressão diastólica para homens (Sexo == 1)
pressao_h = df_2[df_2['Sexo'] == 1]['PressaoDiastolica_1']

# Seleciona os valores de pressão diastólica para mulheres (Sexo == 2)
pressao_m = df_2[df_2['Sexo'] == 2]['PressaoDiastolica_1']

# Realiza uma amostragem de até 1000 valores de pressão diastólica para homens
pressao_h_amostra = pressao_h.sample(n=min(1000, len(pressao_h)), random_state=42)

# Realiza uma amostragem de até 1000 valores de pressão diastólica para mulheres
pressao_m_amostra = pressao_m.sample(n=min(1000, len(pressao_m)), random_state=42)

# Executa o teste de normalidade Shapiro-Wilk para o grupo masculino
shapiro_stat_h, shapiro_p_h = stats.shapiro(pressao_h_amostra)

# Executa o teste de normalidade Shapiro-Wilk para o grupo feminino
shapiro_stat_m, shapiro_p_m = stats.shapiro(pressao_m_amostra)

# Exibe os resultados do teste Shapiro-Wilk para homens
print(f"Shapiro-Wilk (masculino): estatística = {shapiro_stat_h:.3f}, p-valor = {shapiro_p_h:.5f}")

# Exibe os resultados do teste Shapiro-Wilk para mulheres
print(f"Shapiro-Wilk (feminino): estatística = {shapiro_stat_m:.3f}, p-valor = {shapiro_p_m:.5f}")

# Verifica se o p-valor para homens é menor ou igual ao nível de significância
if shapiro_p_h <= alpha:
    print("Rejeita H0 para homens: pressão diastólica não é normal.")
else:
    print("Aceita H0 para homens: pressão diastólica é normal.")

# Verifica se o p-valor para mulheres é menor ou igual ao nível de significância
if shapiro_p_m <= alpha:
    print("Rejeita H0 para mulheres: pressão diastólica não é normal.")
else:
    print("Aceita H0 para mulheres: pressão diastólica é normal.")

# Compara as estatísticas Shapiro-Wilk para identificar qual grupo tem maior desvio da normalidade
if shapiro_stat_h < shapiro_stat_m:
    print("O grupo masculino demonstra maior desvio da normalidade (estatística Shapiro-Wilk menor).")
else:
    print("O grupo feminino demonstra maior desvio da normalidade (estatística Shapiro-Wilk menor).")

### 3. Associação entre Escolaridade e Consumo de Álcool

**Pergunta:** Utilize o teste Qui-Quadrado de Independência para verificar se a Escolaridade (Escolaridade) é independente do consumo de álcool nos últimos 12 meses (ConsumiuAlcool12Meses)?

**O que entregar:**
- tabela de contingência
- estatística χ², p-valor e graus de liberdade
- conclusão sobre independência
- interpretação contextual utilizando o conceito de H0 e HA das páginas 13–15 do slide


In [0]:
#===============================
# Teste Qui-Quadrado de Independência
#===============================

# H0: Escolaridade é independente do consumo de álcool nos últimos 12 meses
# HA: Escolaridade é dependente do consumo de álcool nos últimos 12 meses

# Cria uma cópia do DataFrame original
df_3 = df.copy()

# Remove linhas com valores nulos nas colunas 'Escolaridade' e 'ConsumiuAlcool12Meses'
df_3.dropna(subset=['Escolaridade', 'ConsumiuAlcool12Meses'], inplace=True)

# Cria a tabela de contingência entre Escolaridade e Consumo de Álcool
contingencia = pd.crosstab(df_3['Escolaridade'], df_3['ConsumiuAlcool12Meses'])

# Exibe a tabela de contingência
print("Tabela de contingência:")
print(contingencia)
print('------')

# Realiza o teste Qui-Quadrado de Independência
chi2_stat, chi2_p, chi2_dof, chi2_exp = stats.chi2_contingency(contingencia)

# Exibe a estatística do teste Qui-Quadrado
print(f"\nEstatística χ²: {chi2_stat:.3f}")

# Exibe o p-valor do teste
print(f"p-valor: {chi2_p:.5f}")

# Exibe os graus de liberdade do teste
print(f"Graus de liberdade: {chi2_dof}")

# Verifica se o p-valor é menor ou igual ao nível de significância e interpreta o resultado
if chi2_p <= alpha:
    print("Rejeita H0: Escolaridade é dependente do consumo de álcool nos últimos 12 meses (há associação).")
else:
    print("Aceita H0: Escolaridade é independente do consumo de álcool nos últimos 12 meses (não há associação).")

# Interpretação contextual do resultado do teste
print("\nInterpretação: O teste avalia se existe associação entre escolaridade e consumo de álcool. Se rejeitamos H0, concluímos que escolaridade influencia o consumo de álcool nos últimos 12 meses. Se aceitamos H0, não há evidência de associação entre as variáveis.")

### 4. Teste de Proporções: Proporção de Obesidade entre Fumantes vs Não-Fumantes

**Crie uma variável binária:**
- Obeso = 1 se IMC ≥ 30, caso contrário 0.

**Pergunta:**

Teste se a proporção de obesos é maior entre fumantes do que
entre não fumantes. Utilize um Z-test para diferença de proporções.

**Entregáveis:**

- proporções dos dois grupos
- estatística Z
- p-valor
- conclusão interpretada.

In [0]:
df_4 = df.copy()

# Remove linhas com valores nulos na coluna 'IMC'
df_4.dropna(subset='IMC', inplace=True)

# Cria a variável binária 'Obeso' (1 se IMC >= 30, senão 0)
df_4['Obeso'] = np.where(df_4['IMC'] >= 30, 1, 0)

# Seleciona fumantes (JaFumou100Cigarros == 1)
fumantes = df_4[df_4['JaFumou100Cigarros'] == 1]

# Seleciona não fumantes (JaFumou100Cigarros == 2)
nao_fumantes = df_4[df_4['JaFumou100Cigarros'] == 2]

# Calcula a proporção de obesos entre fumantes
prop_obesos_fumantes = fumantes['Obeso'].mean()

# Calcula a proporção de obesos entre não fumantes
prop_obesos_nao_fumantes = nao_fumantes['Obeso'].mean()

# Realiza o Z-test para diferença de proporções (unilateral à direita)
count = [fumantes['Obeso'].sum(), nao_fumantes['Obeso'].sum()]
nobs = [len(fumantes), len(nao_fumantes)]
z_stat, p_value = proportions_ztest(count, nobs, alternative='larger')

# Exibe a proporção de obesos entre fumantes
print(f"Proporção de obesos entre fumantes: {prop_obesos_fumantes:.3f}")

# Exibe a proporção de obesos entre não fumantes
print(f"Proporção de obesos entre não fumantes: {prop_obesos_nao_fumantes:.3f}")

# Exibe a estatística Z do teste
print(f"Estatística Z: {z_stat:.3f}")

# Exibe o p-valor do teste
print(f"P-valor: {p_value:.5f}")

# Interpreta o resultado do teste de proporções
if p_value < alpha:
    print("Rejeita H0: A proporção de obesos é maior entre fumantes do que entre não fumantes.")
else:
    print("Aceita H0: Não há evidência de que a proporção de obesos seja maior entre fumantes.")

### 5. Amostragem Estratificada Avançada + Análise Inferencial
### 
**Pergunta:** Faça uma amostragem estratificada por faixa etária, definindo os estratos:
- 18–29
- 30–44
- 45–64
- 65+

Extraia uma amostra total de 400 indivíduos proporcional aos tamanhos dos estratos. Em seguida responda : A média do Peso_kg dessa amostra estratificada é estatisticamente igual à média da população adulta inteira?

Realize um t-test simples, formulando:

H0: μ_amostra = μ_população

HA: μ_amostra ≠ μ_população

Interprete o p-valor no contexto da Aula 4, especialmente sobre significância estatística.

In [0]:
# Realiza amostragem estratificada proporcional por faixa etária e compara a média do Peso_kg da amostra com a média da população usando t-test

# Amostragem estratificada por faixa etária e t-test para média do Peso_kg

df_5 = df.copy()
df_5.dropna(subset=['Idade', 'Peso_kg'], inplace=True)

# Define os estratos de faixa etária
bins = [18, 30, 45, 65, np.inf]
labels = ['18-29', '30-44', '45-64', '65+']
df_5['FaixaEtaria'] = pd.cut(df_5['Idade'], bins=bins, labels=labels, right=False)

# Calcula o tamanho de cada estrato
estratos = df_5['FaixaEtaria'].value_counts().sort_index()
proporcoes = estratos / estratos.sum()
n_total = 400
n_estratos = (proporcoes * n_total).round().astype(int)

# Realiza a amostragem estratificada proporcional
amostra_estratificada = pd.concat([
    df_5[df_5['FaixaEtaria'] == faixa].sample(n=n_estratos[faixa], random_state=42)
    for faixa in labels
])

# Média do Peso_kg na amostra estratificada e na população
media_amostra = amostra_estratificada['Peso_kg'].mean()
media_populacao = df_5['Peso_kg'].mean()

# t-test para comparar médias
t_stat, p_value = stats.ttest_1samp(amostra_estratificada['Peso_kg'], media_populacao)

print(f"Média Peso_kg amostra estratificada: {media_amostra:.2f}")
print(f"Média Peso_kg população: {media_populacao:.2f}")
print(f"t estatístico: {t_stat:.3f}")
print(f"p-valor: {p_value:.5f}")

if p_value < alpha:
    print("Rejeita H0: A média do Peso_kg da amostra estratificada é diferente da média da população.")
else:
    print("Aceita H0: Não há evidência de diferença entre as médias do Peso_kg da amostra e da população.")